In [13]:
import torch, numpy as np
from utils import unpack_sym

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ms_file = 'data/feature_engineering_data.h5'


In [ ]:
import sys
sys.path.append('./feature_engineering/')
import feature_extraction as extract
from utils import MicrostructureImageDataset

MS_train = MicrostructureImageDataset(file_path=ms_file, group_name= 'train_set', dset_name='image_data')
MS_bench = MicrostructureImageDataset(file_path=ms_file, group_name= 'benchmark_set', dset_name='image_data')

image_index = 17875
image = MS_train[image_index]

kappa0 = 2.97654        # isotropric thermal heat conductivity of the matrix material
kappa1 = 98.03453       # isotropic thermal heat conductivity of the inclusion material
R = kappa0/kappa1       # Phase contrast ratio

image = np.expand_dims(image.cpu().numpy(), axis=0)
train_feature = extract.full_computation(image) 
train_feature = torch.tensor(np.append(train_feature, [[1/R, R]], axis=1), device=device, dtype=torch.float32)



In [ ]:
import matplotlib.pyplot as plt


model_jit = torch.jit.load('models/vrnn_therm2D_jit.pt')

kappa_pred = unpack_sym(model_jit(train_feature), dim=2)*kappa0

print(kappa_pred.cpu().detach().numpy())


plt.figure(figsize=(4, 4))
plt.imshow(image[0], cmap='viridis')
plt.title(f'Microstructure Image (Index: {image_index})')
plt.axis('equal')
plt.show()
